## **SLIC Superpixel Clustering on Rice Field Heatmaps**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import slic
from skimage.color import label2rgb
from skimage.color import rgb2lab
from PIL import Image
import os
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -n '/content/drive/MyDrive/Satellite - Rice Yield/Self-Supervised Segmentation/Data/archive.zip'

Archive:  /content/drive/MyDrive/Satellite - Rice Yield/archive.zip


In [ ]:
# Load the image
folder_path = '/content/ADEChallengeData2016/images/training'
input_images = sorted(os.listdir(folder_path))
output_path = '/content/drive/MyDrive/Satellite - Rice Yield/Self-Supervised Segmentation/Slic_Merged_Segments'

In [ ]:
threshold = 2

In [ ]:
input_images[0:5]

['ADE_train_00000001.jpg',
 'ADE_train_00000002.jpg',
 'ADE_train_00000003.jpg',
 'ADE_train_00000004.jpg',
 'ADE_train_00000005.jpg']

In [ ]:
# SLIC Superpixel Segmentation Parameters
num_clusters = 300
compactness = 9

In [ ]:
# lis = [1701]

In [ ]:
def track_sch_skips(input_image):
  file_path = '/content/drive/MyDrive/Satellite - Rice Yield/single_ch_skips_new.txt'
  with open(file_path, "a") as f:
    f.write(input_image)

In [ ]:
def smoothen(seg_arr, median_replaced_img):
  for label in np.unique(seg_arr):
    cluster = seg_arr == label
    median_replaced_img[cluster] = np.max(median_replaced_img[cluster], axis=0)

  return median_replaced_img

In [ ]:
def find_extreme_indices(array):
  '''
  Finds the extreme points by 2.a
  '''
  # Find the indices of True elements
  indices = np.argwhere(array)
  # print(indices)
  if len(indices) == 0:
      return None

  # Find the coordinates with extreme values
  leftmost = indices[np.argmin(indices[:, 1])]
  rightmost = indices[np.argmax(indices[:, 1])]
  topmost = indices[np.argmin(indices[:, 0])]
  bottommost = indices[np.argmax(indices[:, 0])]

  return leftmost, rightmost, topmost, bottommost

In [ ]:
def four_extreme_merge(segmentation_arr, median_replaced_img):

  # Make a dictionary containing i, j values such that if we add it to original pixel coodinate we get the outer pixel
  # For example: Outer Point of leftmost point L(i, j) would be a arbitrary point K(i, j - 1)
  # That means if we add (0, -1) to L(i, j) it will give the outer point
  outer_side_dict = {'leftmost': (0, -1), 'rightmost':(0, 1), 'topmost':(-1, 0), 'bottommost': (1, 0)}
  # print(np.unique(segmentation_arr))

  for label in np.unique(segmentation_arr):
    # Individually selects labels and get the boolean array for find_extreme_indices
    cluster = segmentation_arr == label
    indices = find_extreme_indices(cluster)

    if indices is None:
      continue
    else:
      leftmost, rightmost, topmost, bottommost = indices

      for key, value in outer_side_dict.items():
        row, col = eval(key)

        outer = eval(key) + value
        outer_row, outer_col = outer

        try:
          pixel_median = median_replaced_img[row, col]
          neighbor_median = median_replaced_img[outer_row, outer_col]
          neighbor_label = segmentation_arr[outer_row, outer_col]
          pixel_label = segmentation_arr[row, col]

          if abs(neighbor_median - pixel_median) <= threshold:
            segmentation_arr[segmentation_arr == neighbor_label] = pixel_label
            # median_replaced_img[segmentation_arr == pixel_label] = pixel_median

        except IndexError:
          continue

  return segmentation_arr

In [ ]:
counter = 0
for input_image in input_images:
  image_path = os.path.join(folder_path, input_image)
  image_number = re.findall(r'\d+', image_path)

  # if int(image_number[1]) > 14523:
  image = np.array(Image.open(image_path))

  # Skip all the single channel image and keep track of it
  if len(image.shape) != 3:
    counter += 1
    track_sch_skips(input_image)
    continue

  segmentation_array = slic(image, n_segments=num_clusters, compactness=compactness)
  lab_image = rgb2lab(image)[:, :, 0]

  for label in np.unique(segmentation_array):
    cluster = lab_image[segmentation_array == label]
    cluster_median = np.median(cluster, axis=0)

    lab_image[segmentation_array == label] = cluster_median


  # Merge Similar Clusters
  new_segmentation_arr = four_extreme_merge(segmentation_array, lab_image)
  new_median_replaced_image = smoothen(new_segmentation_arr, lab_image)



  # Convert the lab_image to PIL Image
  lab_image_pil = Image.fromarray((new_median_replaced_image * 2.55).astype(np.uint8), mode='L')

  # Save the lab_image
  # image_number = re.findall(r'\d+', image_path)
  image_name = f"SLIC_mask_{image_number[1]}.png"
  save_path = os.path.join(output_path, image_name)

  lab_image_pil.save(save_path)
  if counter % 100 == 0:
    print(counter, end=", ")
  counter += 1

0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 